In [1]:
import pandas as pd
import numpy as np

import os, json
import spotipy
import spotipy.util as util
os.chdir("..")
CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_SECRET_ID')

In [2]:
os.chdir("..")

In [3]:
os.getcwd()

'C:\\Users\\charl\\Python\\Projects\\Spotify_Artist_Recommender'

In [4]:
CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_SECRET_ID')


In [5]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

C:\Users\charl\miniconda3\envs\spotify\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [6]:

playlist = '4rnleEAOdmFAbRcNCgZMpY'

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from typing import List, Dict
import os
# Credit to following for first bit of this https://medium.com/swlh/find-your-favorite-artists-in-spotify-playlists-with-python-420e70abd97b

# Authenticate to Spotify
def authenticate(cliend_id: str, client_secret: str) -> spotipy.client.Spotify:
    sp = spotipy.Spotify(
        client_credentials_manager=SpotifyClientCredentials(
            client_id=cliend_id,
            client_secret=client_secret
        )
    )

    return sp


# Number of tracks available in the playlist
def get_pl_length(sp: spotipy.client.Spotify, pl_uri: str) -> int:
    return sp.playlist_tracks(
        pl_uri,
        offset=0,
        fields="total"
    )["total"]


# Get all the artist info about each track of the playlist.
def get_tracks_artist_info(sp: spotipy.client.Spotify, pl_uri: str) -> List[List[Dict]]:
    artists_info = list()
    # Start retrieving tracks from the beginning of the playlist
    offset = 0
    pl_length = get_pl_length(sp, pl_uri)

    # Playlist track retrieval only fetches 100 tracks at a time, hence\
    # the loop to keep retrieving until we reach the end of the playlist
    while offset != pl_length:
        # Get the next batch of tracks
        pl_tracks = sp.playlist_tracks(
            pl_uri,
            offset=offset,
            fields="items.track"
        )

        # Get the list with the info about the artists of each track from the\
        # latest batch and append it to the running list
        [artists_info.extend(pl_item["track"]["artists"])
            for pl_item in pl_tracks["items"]]

        # Update the offset
        offset += len(pl_tracks["items"])

    return artists_info

In [8]:
CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_SECRET_ID')
# Get a Spotify authenticated instance
sp_instance = authenticate(CLIENT_ID, CLIENT_SECRET)

# Playlist URI to look up
pl_uri = "spotify:playlist:7bLzIyyGRUJw78eHtUZItf"

# Get the artist information for all tracks of the playlist
artists_info = get_tracks_artist_info(sp_instance, pl_uri)

In [9]:
def create_artist_df(sp_instance, pl_uri):

    artist_info = get_tracks_artist_info(sp_instance, pl_uri)
    
    
    df = pd.DataFrame(columns = ["artist", "artist_id", "url"])
    for ind, feats in enumerate(artist_info):
        try:
            artist = feats["name"]
            art_id = feats["id"]
            url = feats["external_urls"]["spotify"]
            df.loc[len(df), :] = [artist, art_id, url]
        except:
            print(feats["name"])
        
    df = df.drop_duplicates(subset=['artist_id']).reset_index(drop=True)
    
    return df

In [31]:
get_tracks_artist_info(sp, "spotify:playlist:4rnleEAOdmFAbRcNCgZMpY")

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2FXC3k01G6Gw61bmprjgqS'},
  'href': 'https://api.spotify.com/v1/artists/2FXC3k01G6Gw61bmprjgqS',
  'id': '2FXC3k01G6Gw61bmprjgqS',
  'name': 'Hozier',
  'type': 'artist',
  'uri': 'spotify:artist:2FXC3k01G6Gw61bmprjgqS'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/2KsP6tYLJlTBvSUxnwlVWa'},
  'href': 'https://api.spotify.com/v1/artists/2KsP6tYLJlTBvSUxnwlVWa',
  'id': '2KsP6tYLJlTBvSUxnwlVWa',
  'name': 'Mike Posner',
  'type': 'artist',
  'uri': 'spotify:artist:2KsP6tYLJlTBvSUxnwlVWa'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/1Bo8Afb2Qbjs4x6kJHyjle'},
  'href': 'https://api.spotify.com/v1/artists/1Bo8Afb2Qbjs4x6kJHyjle',
  'id': '1Bo8Afb2Qbjs4x6kJHyjle',
  'name': 'Gigamesh',
  'type': 'artist',
  'uri': 'spotify:artist:1Bo8Afb2Qbjs4x6kJHyjle'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4V8LLVI7PbaPR0K2TGSxFF'},
  'href': 'https://api.spotify.com/v1/artists

In [11]:
playlist_uris = ["spotify:playlist:7bLzIyyGRUJw78eHtUZItf"]
artist_info = []
for pl in playlist_uris:
    artist_info.extend(get_tracks_artist_info(sp, pl))

df = pd.DataFrame(columns=["artist_name", "artist_id", "url"])
for ind, feats in enumerate(artist_info):
    try:
        artist = feats["name"]
        art_id = feats["id"]
        url = feats["external_urls"]["spotify"]
        df.loc[len(df), :] = [artist, art_id, url]
    except:
        print(feats["name"])

df = df.drop_duplicates(subset=['artist_id']).reset_index(drop=True)


The Amity Affliction
Linkin Park
Bring Me The Horizon
Bring Me The Horizon
The Plot In You
Hollywood Undead
Hollywood Undead
Brockhampton
Brockhampton
The Plot in You
Brockhampton
BROCKHAMPTON
Bring Me The Horizon
Bring Me The Horizon


In [12]:
df = create_artist_df(sp_instance, pl_uri)

The Amity Affliction
Linkin Park
Bring Me The Horizon
Bring Me The Horizon
The Plot In You
Hollywood Undead
Hollywood Undead
Brockhampton
Brockhampton
The Plot in You
Brockhampton
BROCKHAMPTON
Bring Me The Horizon
Bring Me The Horizon


In [13]:
df.head()

,artist,artist_id,url
0,Crown The Empire,2vKiJjsgjgqIECUyYeIVvO,https://open.spotify.com/artist/2vKiJjsgjgqIEC...
1,Secrets,1TRVqozKc5Qk1cjwRdWBSl,https://open.spotify.com/artist/1TRVqozKc5Qk1c...
2,Bring Me The Horizon,1Ffb6ejR6Fe5IamqA5oRUF,https://open.spotify.com/artist/1Ffb6ejR6Fe5Ia...
3,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,https://open.spotify.com/artist/6XyY86QOPPrYVG...
4,The Amity Affliction,6kNKUYGn6VNGsRoXmyoDPK,https://open.spotify.com/artist/6kNKUYGn6VNGsR...


In [14]:
artist_ids = df['artist_id']

In [15]:
from flask_app.utils.data_retriever import FeatureFinder

In [16]:
FF = FeatureFinder(CLIENT_ID, CLIENT_SECRET)

In [17]:
FF.get_artist_song_feats('2vKiJjsgjgqIECUyYeIVvO')

array([ 4.59800e-01,  8.74700e-01, -4.83520e+00,  8.54200e-02,
        9.01456e-02,  5.36047e-03,  2.44830e-01,  3.08300e-01,
        1.29644e+02])

In [18]:
FF._get_top_tracks('2vKiJjsgjgqIECUyYeIVvO')

['14uAFiZ94iMTeEpXq1cLCi',
 '1EQcTEDuJ0tJ0Zi4OKQx9c',
 '1saCzowTZeWnT2N1dEAWIM',
 '7cOLm2VMd1hKadKzw41dmB',
 '10eDxSTjwMBq1ZjZK5b1cK',
 '2wNQm6zXDZTz1xrQbA78A5',
 '5uxHTiFxxUO9l3KSdz44tP',
 '439sdo18d2vVo7crU9KWp1',
 '6h2YXTAX6QN83w0gOTjDZa',
 '0YSZ8vuwoqxS8Eo4jhyZVZ']

In [19]:
FF.sp.audio_features('0YSZ8vuwoqxS8Eo4jhyZVZ')

[{'danceability': 0.525,
  'energy': 0.958,
  'key': 8,
  'loudness': -3.642,
  'mode': 1,
  'speechiness': 0.0653,
  'acousticness': 0.00026,
  'instrumentalness': 0,
  'liveness': 0.391,
  'valence': 0.461,
  'tempo': 120.06,
  'type': 'audio_features',
  'id': '0YSZ8vuwoqxS8Eo4jhyZVZ',
  'uri': 'spotify:track:0YSZ8vuwoqxS8Eo4jhyZVZ',
  'track_href': 'https://api.spotify.com/v1/tracks/0YSZ8vuwoqxS8Eo4jhyZVZ',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0YSZ8vuwoqxS8Eo4jhyZVZ',
  'duration_ms': 251000,
  'time_signature': 4}]

In [33]:

FF._get_song_features(['0YSZ8vuwoqxS8Eo4jhyZVZ'])

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.525,0.958,-3.642,0.0653,0.00026,0,0.391,0.461,120.06


In [21]:
FF.get_artist_song_feats('2vKiJjsgjgqIECUyYeIVvO')

array([ 4.59800e-01,  8.74700e-01, -4.83520e+00,  8.54200e-02,
        9.01456e-02,  5.36047e-03,  2.44830e-01,  3.08300e-01,
        1.29644e+02])

In [22]:
art_feats = []
for i in artist_ids:
    feats = FF.get_artist_song_feats(i)
    art_feats.append(feats)

KeyboardInterrupt: 

In [ ]:
test = pd.DataFrame(art_feats, columns = ["danceability", "energy", "loudness", "speechiness", "acousticness",
                     "instrumentalness", "liveness", "valence", "tempo"])

In [ ]:
new = pd.concat([df, test], axis=1)

In [ ]:
new.dropna(inplace=True)

In [ ]:
new.reset_index(inplace=True, drop=True)

### Let's Try our FeatureFinder class

In [1]:

import pandas as pd
import numpy as np

import os, json
import spotipy
import spotipy.util as util
os.chdir("..")
from flask_app.utils.data_retriever import FeatureFinder
CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_SECRET_ID')

In [26]:
FF = FeatureFinder(CLIENT_ID, CLIENT_SECRET)

In [3]:
new = FF.create_artist_df_with_features(["spotify:playlist:4rnleEAOdmFAbRcNCgZMpY"])

0       2FXC3k01G6Gw61bmprjgqS
1       2KsP6tYLJlTBvSUxnwlVWa
2       1Bo8Afb2Qbjs4x6kJHyjle
3       4V8LLVI7PbaPR0K2TGSxFF
4       1U1el3k54VvEUzo3ybLPlM
                 ...          
2415    5R5MLOQbI4lb7GHYioPqoN
2416    6S0IvKlvPMX1RtAYtVpUV8
2417    3YJMzGjAm245zwJooY7PEj
2418    1sDVuztNhGcetCLChkleDA
2419    20wkVLutqVOYrc0kxFs7rA
Name: artist_id, Length: 2420, dtype: object
1Cs0zKBU1kc0i8ypK3B9ai cannot be found
7pmVkPd2abo5e1SIsDez5p cannot be found
7o95ZoZt5ZYn31e9z1Hc0a cannot be found
3l3sSaPVrP627tNCgt4Tp1 cannot be found


In [9]:
new.dropna(inplace=True)

In [10]:
new.reset_index(inplace=True, drop=True)

In [11]:
new.isnull().sum()

artist_name         0
artist_id           0
url                 0
danceability        0
energy              0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

In [13]:
ST = StandardScaler()

In [44]:
scaled = new.copy()

In [45]:
ST.fit(scaled.iloc[:, 3:])

StandardScaler()

In [46]:
scaled.iloc[:, 3:] = ST.transform(scaled.iloc[:, 3:])

In [47]:
scaled.head()

,artist_name,artist_id,url,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Hozier,2FXC3k01G6Gw61bmprjgqS,https://open.spotify.com/artist/2FXC3k01G6Gw61...,-1.640778,-1.853883,-0.801785,-0.821215,2.270466,0.250408,-0.813122,-1.296883,0.560204
1,Mike Posner,2KsP6tYLJlTBvSUxnwlVWa,https://open.spotify.com/artist/2KsP6tYLJlTBvS...,0.582168,-0.428350,0.369176,-0.688406,0.049203,0.050330,0.349312,0.641593,-1.271879
2,Gigamesh,1Bo8Afb2Qbjs4x6kJHyjle,https://open.spotify.com/artist/1Bo8Afb2Qbjs4x...,0.847450,0.049566,0.078830,-0.247897,-0.664883,0.659661,-0.055162,0.697769,-0.507854
3,"Tyler, The Creator",4V8LLVI7PbaPR0K2TGSxFF,https://open.spotify.com/artist/4V8LLVI7PbaPR0...,-1.056964,-0.876396,-0.914893,0.909258,0.743146,-0.425771,2.664242,-0.458448,-2.062722
4,Kali Uchis,1U1el3k54VvEUzo3ybLPlM,https://open.spotify.com/artist/1U1el3k54VvEUz...,0.159073,-0.563511,-0.308473,-0.186358,0.403324,-0.355890,-0.594514,-0.030377,-0.683846


In [18]:
import joblib

# Next steps are to add this scaling functionality to a new class. Then save it and change my similarity metric to dot product

### Then I will need to add functionality to put a list of playlists in and recceive back a huge artist df. Then I need to actually collect that data


# Maybe I can apply dot product in batch so I don't have to use a for loop

In [19]:
from flask_app.utils.artist_recommender import ArtistRecommender
from datetime import datetime
import numpy as np
from operator import itemgetter
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
AE = ArtistRecommender()

In [21]:
AE.scaler = ST

In [24]:
artist_id, artist_name, url = AE._get_artist_id('Pusha T')

### Current approach (may be slow)

In [27]:
artist_vector = FF.get_artist_song_feats(artist_id)
artist_vector = AE.scaler.transform([artist_vector])



In [29]:
"""
Approach 1
"""
start = datetime.now()

similarity_dic = {}
for i in range(len(scaled)):
    score = cosine_similarity(artist_vector, [scaled.iloc[i, 3:].values])
    similarity_dic[scaled['artist_name'][i]] = score

top_vals = dict(sorted(similarity_dic.items(), key=itemgetter(1), reverse=True)[:10])


end = datetime.now()
print(end-start)

top_vals

0:00:00.813572


{'Pusha T': array([[1.]]),
 'Ca$his': array([[0.98423295]]),
 'T.I.': array([[0.97503612]]),
 'Lloyd Banks': array([[0.97350832]]),
 'Remy Ma': array([[0.97251627]]),
 'Jadakiss': array([[0.96784883]]),
 'Juelz Santana': array([[0.95495659]]),
 'KAMASUTRA 9G': array([[0.94874556]]),
 'Lil Wayne': array([[0.93906666]]),
 'Desiigner': array([[0.93751922]])}

In [30]:
"""
Approach 2
"""
start = datetime.now()

dic = {}
scaled['cosine_sim'] = scaled.iloc[:, 3:].apply(lambda row: cosine_similarity(artist_vector, [row])[0][0], axis=1)
scaled = scaled.sort_values(by = ['cosine_sim'], ascending=False)

dic = {}
for i in range(10):
    row = scaled.iloc[i]
    artist = row['artist_name']
    url = row['url']
    sim = row['cosine_sim']
    dic[artist] = {'url':url,
                   'sim': sim}

scaled.drop(columns = ['cosine_sim'], inplace=True)
end = datetime.now()
print(end-start)

dic

0:00:00.423871


{'Pusha T': {'url': 'https://open.spotify.com/artist/0ONHkAv9pCAFxb0zJwDNTy',
  'sim': 1.0},
 'Ca$his': {'url': 'https://open.spotify.com/artist/3PDMMt0Ip2WHysoAOEAdBq',
  'sim': 0.9842329536427934},
 'T.I.': {'url': 'https://open.spotify.com/artist/4OBJLual30L7gRl5UkeRcT',
  'sim': 0.975036120346118},
 'Lloyd Banks': {'url': 'https://open.spotify.com/artist/3vDUJHQtqT3jFRZ2ECXDTi',
  'sim': 0.9735083171150429},
 'Remy Ma': {'url': 'https://open.spotify.com/artist/39mHYiNmLR7p8PXNG8Wll6',
  'sim': 0.9725162717156532},
 'Jadakiss': {'url': 'https://open.spotify.com/artist/5pnbUBPifNnlusY8kTBivi',
  'sim': 0.9678488261203435},
 'Juelz Santana': {'url': 'https://open.spotify.com/artist/6Uh8uJyN9g7oFjDK16nJgb',
  'sim': 0.9549565936772739},
 'KAMASUTRA 9G': {'url': 'https://open.spotify.com/artist/3hk3Hi3NyolREZ9ykKAmly',
  'sim': 0.9487455582508849},
 'Lil Wayne': {'url': 'https://open.spotify.com/artist/55Aa2cqylxrFIXC767Z865',
  'sim': 0.9390666605938497},
 'Desiigner': {'url': 'https:/

In [ ]:
artist_name

In [ ]:
artist_vector

In [ ]:
scaled[scaled['artist']=='Pusha T']

In [56]:
joblib.dump(ST, "flask_app/MyData/artist_feature_scaler.save")

['flask_app/MyData/artist_feature_scaler.save']

In [48]:
scaled.reset_index(inplace=True, drop=True)


In [49]:
scaled

,artist_name,artist_id,url,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Hozier,2FXC3k01G6Gw61bmprjgqS,https://open.spotify.com/artist/2FXC3k01G6Gw61...,-1.640778,-1.853883,-0.801785,-0.821215,2.270466,0.250408,-0.813122,-1.296883,0.560204
1,Mike Posner,2KsP6tYLJlTBvSUxnwlVWa,https://open.spotify.com/artist/2KsP6tYLJlTBvS...,0.582168,-0.428350,0.369176,-0.688406,0.049203,0.050330,0.349312,0.641593,-1.271879
2,Gigamesh,1Bo8Afb2Qbjs4x6kJHyjle,https://open.spotify.com/artist/1Bo8Afb2Qbjs4x...,0.847450,0.049566,0.078830,-0.247897,-0.664883,0.659661,-0.055162,0.697769,-0.507854
3,"Tyler, The Creator",4V8LLVI7PbaPR0K2TGSxFF,https://open.spotify.com/artist/4V8LLVI7PbaPR0...,-1.056964,-0.876396,-0.914893,0.909258,0.743146,-0.425771,2.664242,-0.458448,-2.062722
4,Kali Uchis,1U1el3k54VvEUzo3ybLPlM,https://open.spotify.com/artist/1U1el3k54VvEUz...,0.159073,-0.563511,-0.308473,-0.186358,0.403324,-0.355890,-0.594514,-0.030377,-0.683846
...,...,...,...,...,...,...,...,...,...,...,...,...
2337,Mindless Self Indulgence,5R5MLOQbI4lb7GHYioPqoN,https://open.spotify.com/artist/5R5MLOQbI4lb7G...,-0.327923,1.077831,0.688251,0.148428,-1.084358,-0.338582,0.822853,1.151438,1.296266
2338,Drax Project,6S0IvKlvPMX1RtAYtVpUV8,https://open.spotify.com/artist/6S0IvKlvPMX1Rt...,0.249114,-0.290203,0.204802,-0.124029,-0.071174,-0.427402,-0.575781,0.272543,-0.125091
2339,Chrisette Michele,3YJMzGjAm245zwJooY7PEj,https://open.spotify.com/artist/3YJMzGjAm245zw...,-0.592237,-1.057108,0.056389,-0.280114,1.668024,-0.427388,-0.056465,-0.640484,-0.949460
2340,Michel Teló,1sDVuztNhGcetCLChkleDA,https://open.spotify.com/artist/1sDVuztNhGcetC...,-0.703578,0.593941,0.265644,-0.556779,1.841551,-0.427404,5.713675,1.284410,0.648573


In [53]:
scaled.to_csv("flask_app/MyData/artist_features.csv", index=False)

In [54]:
scaled.iloc[:, 3:]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-1.640778,-1.853883,-0.801785,-0.821215,2.270466,0.250408,-0.813122,-1.296883,0.560204
1,0.582168,-0.428350,0.369176,-0.688406,0.049203,0.050330,0.349312,0.641593,-1.271879
2,0.847450,0.049566,0.078830,-0.247897,-0.664883,0.659661,-0.055162,0.697769,-0.507854
3,-1.056964,-0.876396,-0.914893,0.909258,0.743146,-0.425771,2.664242,-0.458448,-2.062722
4,0.159073,-0.563511,-0.308473,-0.186358,0.403324,-0.355890,-0.594514,-0.030377,-0.683846
...,...,...,...,...,...,...,...,...,...
2337,-0.327923,1.077831,0.688251,0.148428,-1.084358,-0.338582,0.822853,1.151438,1.296266
2338,0.249114,-0.290203,0.204802,-0.124029,-0.071174,-0.427402,-0.575781,0.272543,-0.125091
2339,-0.592237,-1.057108,0.056389,-0.280114,1.668024,-0.427388,-0.056465,-0.640484,-0.949460
2340,-0.703578,0.593941,0.265644,-0.556779,1.841551,-0.427404,5.713675,1.284410,0.648573


In [55]:
pd.read_csv("flask_app/MyData/artist_features.csv")

,artist_name,artist_id,url,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Hozier,2FXC3k01G6Gw61bmprjgqS,https://open.spotify.com/artist/2FXC3k01G6Gw61...,-1.640778,-1.853883,-0.801785,-0.821215,2.270466,0.250408,-0.813122,-1.296883,0.560204
1,Mike Posner,2KsP6tYLJlTBvSUxnwlVWa,https://open.spotify.com/artist/2KsP6tYLJlTBvS...,0.582168,-0.428350,0.369176,-0.688406,0.049203,0.050330,0.349312,0.641593,-1.271879
2,Gigamesh,1Bo8Afb2Qbjs4x6kJHyjle,https://open.spotify.com/artist/1Bo8Afb2Qbjs4x...,0.847450,0.049566,0.078830,-0.247897,-0.664883,0.659661,-0.055162,0.697769,-0.507854
3,"Tyler, The Creator",4V8LLVI7PbaPR0K2TGSxFF,https://open.spotify.com/artist/4V8LLVI7PbaPR0...,-1.056964,-0.876396,-0.914893,0.909258,0.743146,-0.425771,2.664242,-0.458448,-2.062722
4,Kali Uchis,1U1el3k54VvEUzo3ybLPlM,https://open.spotify.com/artist/1U1el3k54VvEUz...,0.159073,-0.563511,-0.308473,-0.186358,0.403324,-0.355890,-0.594514,-0.030377,-0.683846
...,...,...,...,...,...,...,...,...,...,...,...,...
2337,Mindless Self Indulgence,5R5MLOQbI4lb7GHYioPqoN,https://open.spotify.com/artist/5R5MLOQbI4lb7G...,-0.327923,1.077831,0.688251,0.148428,-1.084358,-0.338582,0.822853,1.151438,1.296266
2338,Drax Project,6S0IvKlvPMX1RtAYtVpUV8,https://open.spotify.com/artist/6S0IvKlvPMX1Rt...,0.249114,-0.290203,0.204802,-0.124029,-0.071174,-0.427402,-0.575781,0.272543,-0.125091
2339,Chrisette Michele,3YJMzGjAm245zwJooY7PEj,https://open.spotify.com/artist/3YJMzGjAm245zw...,-0.592237,-1.057108,0.056389,-0.280114,1.668024,-0.427388,-0.056465,-0.640484,-0.949460
2340,Michel Teló,1sDVuztNhGcetCLChkleDA,https://open.spotify.com/artist/1sDVuztNhGcetC...,-0.703578,0.593941,0.265644,-0.556779,1.841551,-0.427404,5.713675,1.284410,0.648573


In [58]:
result = FF.sp.search(q='artist:' + "Kanye west", type='artist')

In [63]:
result['artists']['items'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
 'followers': {'href': None, 'total': 14034644},
 'genres': ['chicago rap', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
 'id': '5K4W6rqBFWDnAN6FQUkS6x',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8ac8e7bbed98e31a1559',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/b1414e865a16bc0080085ee1901d0bfd19f83fcb',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ac5cb571c194ca5a942b8096618217839b901a76',
   'width': 160}],
 'name': 'Kanye West',
 'popularity': 92,
 'type': 'artist',
 'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}